1.Import Dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
# Set a random seed for deterministic results/reproducability.

In [3]:
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


2.Data Preprocessing

In [4]:
# Instantiate German and English spaCy models.
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [5]:
# To Tokenizes English and german text from a string into a list of strings (tokens):
def tokenize_de(text):
  return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
# To set Field for assigning parameters for Data Preprocessing:
SRC = Field(tokenize = tokenize_de,                         # initialize token as <sos> i.e start of sentence
            init_token = '<sos>', 
            eos_token = '<eos>',                                   # eos_token as <eos> i.e end of sentence
            lower = True)                                          # Lower case

TRG= Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [7]:
# Data Split:
train_data, validation_data, test_data = Multi30k.splits(exts=('.de','.en'),
                                                         fields = (SRC,TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 1.49MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 229kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 216kB/s]


In [8]:
print(f"Number of training examples: {len(train_data.examples)}")                          #length of train_data i.e src len or trg len
print(f"Number of validation examples: {len(validation_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [9]:
print(vars(train_data.examples[1500]))

{'src': ['eine', 'person', 'in', 'einer', 'warnweste', ',', 'die', 'einen', 'lkw', 'schiebt', '.'], 'trg': ['a', 'person', 'in', 'a', 'safety', 'vest', 'pushing', 'a', 'truck', '.']}


In [10]:
# Buliding Vocabulary 
SRC.build_vocab(train_data, min_freq = 2)            # It is essential vocabulary must be build from training set  and not the validation/test set to avoid information leakage in to the model. 
TRG.build_vocab(train_data, min_freq = 2)           # Using the min_freq argument, only allow tokens that appear at least 2 times in training set

In [11]:
print(f"Unique tokens in source (German) vocabulary: {len(SRC.vocab)}")               # input_dim to encoder
print(f"Unique tokens in target (English) vocabulary: {len(TRG.vocab)}")              # output_dim to decoder

Unique tokens in source (German) vocabulary: 7855
Unique tokens in target (English) vocabulary: 5893


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
#BucketIterator :The final step of preparing the data is to create the iterators. These can be iterated on to return a batch of data which will have the PyTorch tensors containing a batch of numericalized src and trg sentences. 
#Numericalized is just a fancy way of saying they have been converted from a sequence of readable tokens to a sequence of corresponding indexes, using the vocabulary.
BATCH_SIZE = 128
train_iterator, valid_iterator , test_iterator = BucketIterator.splits((train_data,validation_data,test_data),batch_size = BATCH_SIZE, device = device,sort_within_batch=True,)

In [14]:
len(train_iterator),len(valid_iterator),len(test_iterator)

(227, 8, 8)

3.Introduction to GRU

a.Note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

b.A slightly more dramatic variation on the LSTM is the Gated Recurrent Unit, or GRU, introduced by Cho, et al. (2014). It combines the forget and input gates into a single “update gate.” It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models, and has been growing increasingly popular.

c.We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.


4.Seq2Seq (Single layer) GRU Model Building

In [15]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, dropout):
    super().__init__()

    self.hid_dim = hid_dim                                             # Define hidden_state
    self.embedding = nn.Embedding(input_dim,emb_dim)                   # Embedding layer
    self.rnn = nn.GRU(emb_dim,hid_dim)                                 # no dropout as single layer GRU is used

    self.dropout = nn.Dropout(dropout)

  def forward(self, src):                                              
                                                                      # src = [src len, batch size]
    embedded = self.dropout(self.embedding(src))                      # embedded = [src len, batch size, emb dim]
                                                                      #outputs = [src len, batch size, hid dim * n directions] #hidden = [n layers * n directions, batch size, hid dim]
    outputs, (hidden) = self.rnn(embedded)                            #no cell state

    return hidden

class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, dropout):
    super(). __init__()

    self.hid_dim = hid_dim
    self.output_dim = output_dim

    self.embedding = nn.Embedding(output_dim,emb_dim)
    
    self.rnn = nn.GRU(emb_dim+hid_dim, hid_dim)

    self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)

    self.dropout = nn.Dropout(dropout)
 
  def forward(self, input, hidden, context):
    
    input = input.unsqueeze(0)

    embedded = self.dropout(self.embedding(input))

    emb_con = torch.cat((embedded,context),dim=2)

    output,hidden = self.rnn(emb_con, hidden)

    output=torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim=1)

    prediction = self.fc_out(output)

    return prediction, hidden
# Decoder unit consist of one embedding layer , one GRU layer and one dense layer(linear).Embedding layer is getting input of target token and that can be converted into dense representation with emb_dim.
# GRU layer is getting input of emb_dim and two hid_dim i.e context hidden state and previous hidden state.So emb_dim is concatenated with hid_dim of context before get into GRU cell of all time step.
#Linear layer is where output token is getting predicted receiving new hidden state i.e hid_dim*2 from GRU cell , hid_dim from context and emb_dim of target token from embedding layer. 

class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    assert encoder.hid_dim == decoder.hid_dim, \
        "Hidden dimensions of encoder and decoder must be equal!"
 
  def forward(self, src, trg, teacher_forcing_ratio = 0.5):

    batch_size = trg.shape[1]                                     #As trg.shape is [trg len , batch_size]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim                      #len(en.vocab)

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)    #tensor to store decoder outputs # simply prediction token.

    context = self.encoder(src)            #last hidden state of the encoder is context

    hidden = context                       #context also used as the initial hidden state of the decoder

    input = trg[0,:]                       #first input to the decoder is the <sos> tokens

    for t in range(1, trg_len):
     
      output, hidden = self.decoder(input, hidden, context)    #insert input token embedding, previous hidden state and the context state  #receive output tensor (predictions) and new hidden state

      outputs[t] = output                                      #place predictions in a tensor holding predictions for each token

      teacher_force = random.random() < teacher_forcing_ratio  #decide if we are going to use teacher forcing or not
      
      top1 = output.argmax(1)                                  #get the highest predicted token from our predictions

      input = trg[t] if teacher_force else top1                #if teacher forcing, use actual next token as next input #if not, use predicted token

    return outputs

5.Training parameters of Seq2Seq GRU model:

In [18]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM,ENC_EMB_DIM, HID_DIM,ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM,DEC_EMB_DIM,HID_DIM,DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2Seq(enc, dec, device).to(device)

In [19]:
#Weight initialization:
#The parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
#Even though we only have a single layer RNN for our encoder and decoder we actually have more parameters than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,220,293 trainable parameters


In [21]:
#Optimizer
optimizer = optim.Adam(model.parameters())

In [22]:
# initialize the loss function, making sure to ignore the loss on <pad> tokens
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

6.Create Training and Evaluation loop

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()                         #set the model into training mode
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src                   #get the source and target sentences from the batch
        trg = batch.trg
        
        optimizer.zero_grad()            #zero the gradients calculated from the last batch
        
        output = model(src, trg)         #feed the source and target into the model to get the output
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)    #has to eliminate fist element in both target and output tensor as it is <sos> and 0 respectively.Using'view' it can be done
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)              #apply loss function
        
        loss.backward()                            #calculate the loss while back propagation
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip) #clip is used  to prevent gradients from exploding (a common issue in RNNs)
        
        optimizer.step()    #update the parameters of our model by doing an optimizer step
        
        epoch_loss += loss.item()  #sum the loss value to a running total
        
    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    model.eval()   #set the mode to evaluation. This will turn off dropout 
    
    epoch_loss = 0
    
    with torch.no_grad():  #use the with torch.no_grad() block to ensure no gradients are calculated within the block. This reduces memory consumption and speeds things up 
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0)      # 0 indicates turn off teacher forcing  

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [25]:
#create a function that tell that how long an epoch takes.
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
#At each epoch, we'll be checking if our model has achieved the best validation loss so far. If it has, we'll update our best validation loss and save the parameters of our model (called state_dict in PyTorch).
#Then, when we come to test our model, we'll use the saved parameters used to achieve the best validation loss.

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model,train_iterator,optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'Pytorch Enc-Dec Model-GRU.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

# Perplexity is the multiplicative inverse of the probability assigned to the test set by the language model, normalized by the number of words in the test set. If a language model can predict unseen words from the test set.
#As a result, better language models will have lower perplexity values or higher probability values for a test set.

Epoch: 01 | Time: 13m 43s
	Train Loss: 2.676 | Train PPL:  14.525
	 Val. Loss: 3.637 |  Val. PPL:  37.972
Epoch: 02 | Time: 13m 43s
	Train Loss: 2.412 | Train PPL:  11.155
	 Val. Loss: 3.569 |  Val. PPL:  35.498
Epoch: 03 | Time: 13m 39s
	Train Loss: 2.196 | Train PPL:   8.986
	 Val. Loss: 3.559 |  Val. PPL:  35.138
Epoch: 04 | Time: 13m 42s
	Train Loss: 2.028 | Train PPL:   7.595
	 Val. Loss: 3.559 |  Val. PPL:  35.120
Epoch: 05 | Time: 13m 40s
	Train Loss: 1.877 | Train PPL:   6.537
	 Val. Loss: 3.621 |  Val. PPL:  37.370
Epoch: 06 | Time: 13m 39s
	Train Loss: 1.757 | Train PPL:   5.798
	 Val. Loss: 3.616 |  Val. PPL:  37.191
Epoch: 07 | Time: 13m 42s
	Train Loss: 1.673 | Train PPL:   5.327
	 Val. Loss: 3.596 |  Val. PPL:  36.445
Epoch: 08 | Time: 13m 43s
	Train Loss: 1.557 | Train PPL:   4.746
	 Val. Loss: 3.669 |  Val. PPL:  39.228
Epoch: 09 | Time: 13m 48s
	Train Loss: 1.463 | Train PPL:   4.321
	 Val. Loss: 3.669 |  Val. PPL:  39.204
Epoch: 10 | Time: 13m 42s
	Train Loss: 1.398 |

In [28]:
#We'll load the parameters (state_dict) that gave our model the best validation loss and run it the model on the test set.
model.load_state_dict(torch.load('Pytorch Enc-Dec Model-GRU.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


| Test Loss: 3.479 | Test PPL:  32.420 |
